#### Function for read WhatsApp file


In [78]:
def read_file():
    file_name = input('Enter the file path and name: ')
    fhand = open(file_name, 'r', encoding='utf-8')
    return fhand
fhand = read_file()

Enter the file path and name: �צ'אט WhatsApp עם מבוא למכונות.txt


#### Function which is building a dictionary of WhatsApp messages and metadata and export it to json file text


In [79]:
fhand.seek(0,0)
import re
import json

def Whatsapp_messages (fhand):
    
    ## 1 makes the phones and names anonymuos- build a dictionary with id and number or name
    
    id_dic = dict()
    id = 1
    persons_list = list()
    date_format='(([0-9]{2}|[0-9])\.([0][1-9]|[1][0-2]|[0-9])\.[0-9]{4})'
    time_format='([0-9]|[0-9]{2}):[0-9]{2}'
    
    for line in fhand:
         if ((line.find("נוצרה על ידי")==-1) and (line.find("שינה/תה")==-1) and (line.find("החליף/ה")==-1) and (line.find("עזב/ה")==-1) and (line.find("מוצפנות מקצה-לקצה")==-1) and (line.find("קישור ההזמנה")==-1)):
            if ((line.find("הוסיף/ה")==-1) ):
                if (re.search(date_format, line)):
                    makaf = line.find('-')
                    nekudotaim = line.find(':',makaf)
                    result = line[makaf+2:nekudotaim]
                    if not result in persons_list:
                        persons_list.append(result)
                        id_dic[result]= id
                        id = id+1
  
    ## 2 list of dictionaries for WhatsApp messages

    fhand.seek(0,0)
    messages = list()
    for line in fhand:
        line = line.rstrip()
        if ((line.find("נוצרה על ידי")==-1) and (line.find("שינה/תה")==-1) and (line.find("החליף/ה")==-1) and (line.find("עזב/ה")==-1) and (line.find("מוצפנות מקצה-לקצה")==-1) and (line.find("קישור ההזמנה")==-1)):
            if ((line.find("הוסיף/ה")==-1) ):
                if (re.search(date_format, line)):
                    message_dic = dict()
                    makaf = line.find('-')
                    message_dic['datetime']= line[:makaf]
                    nekudotaim = line.find(':',makaf)
                    person = line[makaf+2:nekudotaim]
                    message_dic['id'] = id_dic[person]
                    text = line[nekudotaim+1 :]
                    message_dic['text'] = text
                    messages.append(message_dic)   
                else:
                    text = text + " " + line
                    message_dic['text'] = text 

    ## 3 metadata dictionary for WhatsApp group
    
    fhand.seek(0,0)
    metadata = dict()
    for line in fhand:
        line = line.rstrip()
        if (line.find("נוצרה על ידי")!=-1):
            group = line.find('הקבוצה')
            created = line.find('נוצרה', group)
            metadata['chat_name']= line[group+8:created-2]
            makaf = line.find('-')
            metadata['creation_date']= line[:makaf]
            metadata['num_of_participants']= id
            metadata['creator'] = line[created+13:]

    ## 4 Dictionary for messages and metadata
    
    milon = dict()
    milon['messages'] = messages
    milon['metadata'] = metadata

    ## 5 save the dictionary to json
    
    json_file = metadata['chat_name']+".txt"
    with open (json_file, 'w' , encoding='utf-16le') as outfile:
        json.dump(milon, outfile, ensure_ascii=False)

#### Run the program 

In [80]:
Whatsapp_messages(fhand)

#### Assumptions


In [71]:
#assumptions:
#הצ'אט הוא מאנדרואיד
#כמות המשתתפים בקבוצה נספרה על ידי כמות האנשים שאי פעם שלחו הודעה אחת לפחות בקבוצה
#לשם האסתטיות הפכתי את הבאקסלאש אן בסוף כל שורה לרווח על מנת להימנע מתווים אלו בפלט ההודעות